In [5]:
import sys
import os
import polars as pl

# Add project root to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''), '..')))

from AzureBlobLoader import AzureBlobLoader
from utils.config import CONFIG, CONN_STRING_AZURE_STORAGE

def load_data():
    try:
        loader = AzureBlobLoader(CONFIG, CONN_STRING_AZURE_STORAGE)
        dataframes = loader.load_dataframes()
        print("Data loaded successfully!")
        return dataframes
    except Exception as e:
        print(f"Error loading data: {e}", exc_info=True)

In [6]:
dataframes = load_data()

Successfully read blob from gold/epl_news/dim_date.parquet
Successfully read blob from gold/epl_news/article.parquet
Successfully read blob from gold/epl_news/dim_team.parquet
Successfully read blob from gold/epl_news/reaction.parquet
Successfully read blob from gold/epl_news/dim_sentiment.parquet
Successfully read blob from gold/epl_news/fact_reaction.parquet
Successfully read blob from gold/epl_news/fact_title.parquet
Data loaded successfully!


In [25]:
df_article = dataframes['df_article']
print(df_article.shape)
df_article = df_article.filter(pl.col("article_id").is_not_null())
print(df_article.shape)
df_article = df_article.unique()
print(df_article.shape)
df_article.head()

(198, 4)
(191, 4)
(191, 4)


article_id,fk_team_id,article_title,published_at
str,i64,str,date
"""9000289e879afec1""",5,"""Brighton's extreme approach un…",2024-09-28
"""2ddc5c0fc6b1f57e""",6,"""'The going has got tough - wil…",2024-10-08
"""07b82732a0852ce7""",19,"""Today's trivia challenge""",2024-08-16
"""4c464c176e47df86""",4,"""Tottenham 3-1 Brentford: Bees …",2024-09-21
"""2f6e34717e3d6518""",20,"""Too soon to worry about Wolves…",2024-09-15


In [19]:
df_article.filter(pl.col("article_id").is_null())

article_id,fk_team_id,article_title,published_at
str,i64,str,date
null,16,"""No title found""",null
null,16,"""No title found""",null
null,16,"""No title found""",null
null,16,"""No title found""",null
null,16,"""No title found""",null
null,16,"""No title found""",null
null,1,"""No title found""",null


In [38]:
df_team = dataframes['df_team']
df_team.head()

team_id,team_name
i64,str
1,"""AFC Bournemouth"""
2,"""Arsenal"""
3,"""Aston Villa"""
4,"""Brentford"""
5,"""Brighton & Hove Albion"""


In [41]:
df_nb_articles = df_article \
    .select(["fk_team_id", "article_id"]) \
    .group_by("fk_team_id") \
    .n_unique() \
    .rename({"article_id": "Count"}) \
    .join(df_team, left_on="fk_team_id", right_on="team_id", how="left") \
    .drop("fk_team_id") \
    .sort("Count")

df_nb_articles

Count,team_name
u32,str
5,"""Aston Villa"""
6,"""Crystal Palace"""
7,"""Tottenham Hotspur"""
7,"""Leicester City"""
8,"""Fulham"""
…,…
11,"""Newcastle United"""
12,"""AFC Bournemouth"""
12,"""Chelsea"""
